In [302]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.metrics import r2_score
#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

In [304]:
housing = pd.read_csv("Housing.csv")

In [306]:
housing.head()


,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [308]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price     546 non-null    float64
 1   lotsize   546 non-null    int64  
 2   bedrooms  546 non-null    int64  
 3   bathrms   546 non-null    int64  
 4   stories   546 non-null    int64  
 5   driveway  546 non-null    object 
 6   recroom   546 non-null    object 
 7   fullbase  546 non-null    object 
 8   gashw     546 non-null    object 
 9   airco     546 non-null    object 
 10  garagepl  546 non-null    int64  
 11  prefarea  546 non-null    object 
dtypes: float64(1), int64(5), object(6)
memory usage: 51.3+ KB


In [310]:
X = housing[['driveway']]
y = housing['price']

In [312]:
# one hot encoding
# .get_dummies() from pandas

In [314]:
dum_X = pd.get_dummies(X, drop_first =True)
dum_X

,driveway_yes
0,True
1,True
2,True
3,True
4,True
...,...
541,True
542,True
543,True
544,True


In [316]:
lr = LinearRegression()
lr.fit(dum_X,y)
lr.intercept_,lr.coef_

(48555.77922077922, array([22778.11630161]))

In [318]:
housing.groupby('driveway')['price'].mean()

driveway
no     48555.779221
yes    71333.895522
Name: price, dtype: float64

In [320]:
X = housing[['driveway']]
y = housing['price']

In [322]:
dum_X = pd.get_dummies(X, drop_first =False)
dum_X

,driveway_no,driveway_yes
0,False,True
1,False,True
2,False,True
3,False,True
4,False,True
...,...,...
541,False,True
542,False,True
543,False,True
544,False,True


In [324]:
dum_X = dum_X.drop(columns = 'driveway_yes', axis = 1)

In [326]:
dum_X.head(2)

,driveway_no
0,False
1,False


In [328]:
lr.fit(dum_X,y)
lr.intercept_,lr.coef_

(71333.89552238806, array([-22778.11630161]))

In [330]:
#using 'OneHotEncoding()' from sklearn 

In [332]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output = False, drop = 'first').set_output(transform = 'pandas') # to change it into pandas

In [334]:
dum_X = ohe.fit_transform(X) #will return numpy array by default

In [336]:
lr = LinearRegression()
lr.fit(dum_X,y)
lr.intercept_,lr.coef_

(48555.77922077922, array([22778.11630161]))

In [338]:
#Two columns  'driveway' and 'airco'



In [340]:
X = housing[['driveway','airco']]
y = housing['price']

In [342]:
ohe = OneHotEncoder(sparse_output = False, drop = 'first').set_output(transform = 'pandas') # to change it into pandas

In [344]:
dum_X = ohe.fit_transform(X)

In [352]:
X_train, X_test, y_train, y_test = train_test_split(dum_X,y,random_state=24,test_size=0.3)

In [360]:
lr = LinearRegression()
lr.fit(dum_X,y)
lr.intercept_,lr.coef_

(43790.67032031879, array([19302.6687155, 24460.8923557]))

In [358]:
lr = LinearRegression()
lr.fit(X_train,y_train)
lr.intercept_,lr.coef_

(43843.26871530294, array([19153.77091358, 25517.1951943 ]))

In [362]:
y_pred = lr.predict(X_test)
r2_score(y_pred,y_test)

-1.8749689921229291

In [350]:
dum_X.columns

Index(['driveway_yes', 'airco_yes'], dtype='object')

In [206]:
#with numeric variable

In [208]:
housing['bedrooms'].unique()

array([3, 2, 4, 1, 5, 6], dtype=int64)

In [210]:
X = housing[['driveway','airco','bedrooms']]
y = housing['price']

In [212]:
dum_X = pd.get_dummies(X,drop_first = True)
dum_X


,bedrooms,driveway_yes,airco_yes
0,3,True,False
1,2,True,False
2,3,True,False
3,3,True,False
4,2,True,False
...,...,...,...
541,3,True,True
542,3,True,True
543,3,True,True
544,3,True,True


In [214]:
#column transformer

In [216]:
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

In [218]:
str_cols = list(housing.columns[housing.dtypes==object])
num_cols = list(housing.columns[housing.dtypes!=object])
ohe = OneHotEncoder(sparse_output = False, drop = 'first').set_output(transform = 'pandas')

In [220]:
ct = make_column_transformer(('passthrough',num_cols),(ohe,str_cols),verbose_feature_names_out = False).set_output(transform='pandas')



In [222]:
ct.fit_transform(housing)

,price,lotsize,bedrooms,bathrms,stories,garagepl,driveway_yes,recroom_yes,fullbase_yes,gashw_yes,airco_yes,prefarea_yes
0,42000.0,5850,3,1,2,1,1.0,0.0,1.0,0.0,0.0,0.0
1,38500.0,4000,2,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0
2,49500.0,3060,3,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0
3,60500.0,6650,3,1,2,0,1.0,1.0,0.0,0.0,0.0,0.0
4,61000.0,6360,2,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
541,91500.0,4800,3,2,4,0,1.0,1.0,0.0,0.0,1.0,0.0
542,94000.0,6000,3,2,4,0,1.0,0.0,0.0,0.0,1.0,0.0
543,103000.0,6000,3,2,4,1,1.0,1.0,0.0,0.0,1.0,0.0
544,105000.0,6000,3,2,2,1,1.0,1.0,0.0,0.0,1.0,0.0


In [224]:
#using make_column_selector(dtype_exclude = ) & dtype_include= 

ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),
                             (ohe,make_column_selector(dtype_include = object) ),
                             verbose_feature_names_out = False).set_output(transform='pandas')

ct.fit_transform(housing)

,price,lotsize,bedrooms,bathrms,stories,garagepl,driveway_yes,recroom_yes,fullbase_yes,gashw_yes,airco_yes,prefarea_yes
0,42000.0,5850,3,1,2,1,1.0,0.0,1.0,0.0,0.0,0.0
1,38500.0,4000,2,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0
2,49500.0,3060,3,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0
3,60500.0,6650,3,1,2,0,1.0,1.0,0.0,0.0,0.0,0.0
4,61000.0,6360,2,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
541,91500.0,4800,3,2,4,0,1.0,1.0,0.0,0.0,1.0,0.0
542,94000.0,6000,3,2,4,0,1.0,0.0,0.0,0.0,1.0,0.0
543,103000.0,6000,3,2,4,1,1.0,1.0,0.0,0.0,1.0,0.0
544,105000.0,6000,3,2,2,1,1.0,1.0,0.0,0.0,1.0,0.0


In [226]:
# using hot encoding with Train test split

In [228]:
ohe = OneHotEncoder(sparse_output = False, drop = 'first').set_output(transform = 'pandas')

In [230]:
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),(ohe,make_column_selector(dtype_include = object) ),verbose_feature_names_out = False).set_output(transform='pandas')


In [232]:
X =housing.drop('price',axis =1)
y = housing['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 24, test_size = 0.3)

In [234]:
X_ohe_trn = ct.fit_transform(X_train)
X_ohe_tst = ct.transform(X_test)
lr.fit(X_ohe_trn, y_train)
y_pred = lr.predict(X_ohe_tst)
r2_score(y_test, y_pred)

0.6246856191453717

In [236]:
X =housing.drop('price',axis =1)
y = housing['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 24, test_size = 0.3)

In [238]:

ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),(ohe,make_column_selector(dtype_include = object) ),verbose_feature_names_out = False).set_output(transform='pandas')


In [240]:
#using Pipeline
lr = LinearRegression()
pipe = Pipeline([('CT',ct),('LR',lr)])

In [242]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
r2_score(y_test,y_pred)

0.6246856191453717

In [244]:
# OneHotEncoding +  polynomial Transformation + linearRegression

In [246]:
poly = PolynomialFeatures(degree = 2, include_bias = False).set_output(transform='pandas')

In [248]:
pipe = Pipeline([('CT',ct),('POLY',poly),('LR',lr)])
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
r2_score(y_test,y_pred)

0.5558314095911471

In [366]:
#Grid Search implementation




elastic = ElasticNet()
pipe_el = Pipeline([('CT',ct),('EL',elastic)])

pipe_el.get_params()

{'memory': None,
 'steps': [('CT',
   ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000001B786A2B1D0>),
                                   ('onehotencoder',
                                    OneHotEncoder(drop='first',
                                                  sparse_output=False),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000001B786A2AC30>)],
                     verbose_feature_names_out=False)),
  ('EL', ElasticNet())],
 'verbose': False,
 'CT': ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001B786A2B1D0>),
                                 ('onehotencoder',
                                  OneHotEncoder(drop='first',
                                                sp

In [270]:
#X_ohe_trn = ct.fit_transform(X_train)
#X_ohe_tst = ct.transform(X_test)

In [368]:
params = {'EL__alpha':np.linspace(0.0001,10,20),'EL__l1_ratio': np.linspace(0.0001,1,10)}
gcv_el = GridSearchCV(pipe_el, param_grid = params, verbose = 2) #to display each iteration value -> verbose it  updates what's going on currently
gcv_el.fit(X,y)# verbose > 3 will give all info()

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV] END ..............EL__alpha=0.0001, EL__l1_ratio=0.0001; total time=   0.0s
[CV] END ..............EL__alpha=0.0001, EL__l1_ratio=0.0001; total time=   0.0s
[CV] END ..............EL__alpha=0.0001, EL__l1_ratio=0.0001; total time=   0.0s
[CV] END ..............EL__alpha=0.0001, EL__l1_ratio=0.0001; total time=   0.0s
[CV] END ..............EL__alpha=0.0001, EL__l1_ratio=0.0001; total time=   0.0s
[CV] END .EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001; total time=   0.0s
[CV] END .EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001; total time=   0.0s
[CV] END .EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001; total time=   0.0s
[CV] END .EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001; total time=   0.0s
[CV] END .EL__alpha=0.0001, EL__l1_ratio=0.11120000000000001; total time=   0.0s
[CV] END ..............EL__alpha=0.0001, EL__l1_ratio=0.2223; total time=   0.0s
[CV] END ..............EL__alpha=0.0001, EL__

GridSearchCV(estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001B786A2B1D0>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001B786A2AC30>)],
                                                          verbos...
       2.10534211e+00, 2.63165263e+00, 3.15796316e+00, 3.68427368e+00,
       4.21058421e+00, 4.73689474e+00, 5.26320526e+00, 5.78951579e+00,
       6.31582632e+00, 6.84213684e+00, 7.36844737e+00, 7.89475789e+00,
       8.42106842e+00, 8.94737895e+00, 9.47368947e+00, 1.00000000e+01]),
                         'EL__l1_ratio': array([1.000e-04, 1.112e-01, 2.223e-01, 3.334e-01, 4.445e-01, 5.556e-01,
       6.667e-01, 7.778e-01, 8.889e-01, 1.000e+00])},
             verbose=2)

In [293]:
print("best Params:",gcv_el.best_params_)
print("Best Score:",gcv_el.best_score_)

best Params: {'EL__alpha': 10.0, 'EL__l1_ratio': 1.0}
Best Score: 0.5084207009081492


In [370]:
df_results = pd.DataFrame(gcv_el.cv_results_)
df_results.shape
df_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_EL__alpha,param_EL__l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011111,0.007570,0.005401,0.002333,0.0001,0.0001,"{'EL__alpha': 0.0001, 'EL__l1_ratio': 0.0001}",-0.207801,0.183477,-0.212687,-0.331999,0.311712,-0.051459,0.251487,15
1,0.006717,0.000394,0.004203,0.000400,0.0001,0.1112,"{'EL__alpha': 0.0001, 'EL__l1_ratio': 0.111200...",-0.207808,0.183447,-0.212713,-0.331979,0.311720,-0.051467,0.251484,16
2,0.006509,0.000450,0.004198,0.000397,0.0001,0.2223,"{'EL__alpha': 0.0001, 'EL__l1_ratio': 0.2223}",-0.207816,0.183417,-0.212740,-0.331960,0.311729,-0.051474,0.251480,18
3,0.008311,0.002185,0.005401,0.002798,0.0001,0.3334,"{'EL__alpha': 0.0001, 'EL__l1_ratio': 0.333400...",-0.207824,0.183387,-0.212766,-0.331940,0.311738,-0.051481,0.251477,20
4,0.006197,0.000401,0.004003,0.000007,0.0001,0.4445,"{'EL__alpha': 0.0001, 'EL__l1_ratio': 0.4445}",-0.207831,0.183357,-0.212792,-0.331921,0.311746,-0.051488,0.251474,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.005603,0.000492,0.003399,0.000490,10.0,0.5556,"{'EL__alpha': 10.0, 'EL__l1_ratio': 0.5556}",-0.425227,-0.166936,-0.327999,-0.804561,-0.154794,-0.375903,0.237079,159
196,0.005800,0.000401,0.003508,0.000448,10.0,0.6667,"{'EL__alpha': 10.0, 'EL__l1_ratio': 0.66670000...",-0.417265,-0.156956,-0.318037,-0.795637,-0.143031,-0.366185,0.237878,141
197,0.006401,0.000489,0.003602,0.000492,10.0,0.7778,"{'EL__alpha': 10.0, 'EL__l1_ratio': 0.7778}",-0.402453,-0.138172,-0.299689,-0.778730,-0.120953,-0.347999,0.239335,114
198,0.005606,0.000487,0.003400,0.000494,10.0,0.8889,"{'EL__alpha': 10.0, 'EL__l1_ratio': 0.8889}",-0.365434,-0.089795,-0.255076,-0.734490,-0.064524,-0.301864,0.242775,79


In [295]:
df_results.to_csv("cvhousing_results.csv",index = False)